### 下載圖片
使用`bing_image_downloader`下載指定人物的頭像

In [ ]:
from bing_image_downloader import downloader
downloader.download("Mookie Betts Headshot", limit=300, filter="photo" ,verbose=False)
downloader.download("Freddie Freeman Headshot", limit=300, filter="photo" ,verbose=False)
downloader.download("Shohei Ohtani Headshot", limit=300, filter="photo" ,verbose=False)

### 擷取臉部並篩除相似的影像

In [ ]:
from collect import capture, filter_similar_images

for player in ["Mookie Betts", "Freddie Freeman", "Shohei Ohtani"]:
    capture(player, f"dataset/{player} Headshot", 300)
    filter_similar_images(f"faces/{player}")

### 轉成HOG後紀錄影像和特徵數

In [ ]:
import os
import cv2
from skimage.feature import hog

for player in ["Mookie Betts", "Freddie Freeman", "Shohei Ohtani"]:
    os.makedirs(f"hogs/{player}", exist_ok=True)
    for file in os.listdir(f"faces/{player}"):
        img = cv2.imread(f"faces/{player}/{file}", cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (128, 128))
        features, hog_img = hog(
            img, orientations=9, pixels_per_cell=(8, 8),
            cells_per_block=(2, 2), block_norm='L2-Hys',
            transform_sqrt=True, visualize=True
        )
        # 將值映射到 0 到 255 之間，然後轉換為 uint8，以正確儲存影像
        hog_img = cv2.normalize(hog_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX).astype("uint8")
        cv2.imwrite(f"hogs/{player}/{file}", hog_img)
print("HOG後的特徵數:", len(features))

### 建立一個影像增強版本的資料集

In [ ]:
import os
from collect import augment_image

# 用powershell指令複製faces到augs
!powershell Copy-Item -Path "faces" -Destination "augs" -Recurse
for player in ["Mookie Betts", "Freddie Freeman", "Shohei Ohtani"]:
    for file in os.listdir(f"augs/{player}"):
        file_path = os.path.join(f"augs/{player}", file)
        augment_image(file_path)

### 模型訓練

In [ ]:
!python train.py

### 用影片測試模型結果

In [ ]:
!python detect.py train/1/model.pkl --source "test.mp4"